In [1]:
from datetime import date
import pandas as pd

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 5)

df = catalog.load('raw/openaire/researchproduct_dev#parquet')

[09/08/25 12:12:27] INFO     Loading data from raw/openaire/researchproduct_dev#parquet         ]8;id=463212;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=208661;file:///home/pablo/dev/conicet/kedro-conicet/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\
                             (ParquetDataset)...                                                                   

In [2]:
df

,authors,openAccessColor,publiclyFunded,type,language,countries,subjects,mainTitle,subTitle,descriptions,publicationDate,publisher,embargoEndDate,sources,formats,contributors,coverages,bestAccessRight,container,documentationUrls,codeRepositoryUrl,programmingLanguage,contactPeople,contactGroups,tools,size,version,geoLocations,id,originalIds,pids,dateOfCollection,lastUpdateTimeStamp,indicators,projects,organizations,communities,collectedFrom,instances,isGreen,isInDiamondJournal,relOrganizationId,load_datetime
0,"[{'fullName': 'Susevich, María Laura', 'name':...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Dicistrovirus from the pollinator community fo...,None,[Los Dicistrovirus son una familia de virus qu...,2022-08-31,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::319dc88111c9b2d6021228590e79130a,[50|4dc99724cf04::319dc88111c9b2d6021228590e79...,None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLPAM', 'id': 'openorgs____::db...","[{'code': 'knowmad', 'label': 'Knowmad Institu...",[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
1,"[{'fullName': 'Steffen, Kevin Denis', 'name': ...",gold,False,publication,"{'code': 'spa', 'label': 'Spanish; Castilian'}",None,"[{'provenance': None, 'subject': {'scheme': 'k...",Variability in the growth rates of Saanen kids...,None,[This study was carried out to determine the v...,2023-12-18,"Universidad Nacional Mayor de San Marcos, Facu...",None,[Revista de Investigaciones Veterinarias del P...,[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...","{'conferenceDate': None, 'conferencePlace': No...",None,None,None,None,None,None,None,None,None,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,"[oai:ojs.csi.unmsm:article/25240, 50|4dc99724c...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,[{'acronym': 'Universidad Nacional de La Pampa...,None,[{'key': 'openaire____::0b74b6a356bbf23c245f9a...,"[{'accessRight': {'code': 'c_abf2', 'label': '...",False,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,"[{'fullName': 'Seré, Pablo Ricardo', 'name': '...",None,False,publication,"{'code': 'esl/spa', 'label': 'Spanish'}","[{'code': 'AR', 'label': 'Argentina', 'provena...","[{'provenance': None, 'subject': {'scheme': 'k...",Efecto de la concentración del silano MTMO sob...,None,[Los pretratamientos superficiales a base de s...,2017-01-01,None,None,"[reponame:SEDICI (UNLP), instname:Universidad ...",[application/pdf],None,None,"{'code': 'c_abf2', 'label': 'OPEN', 'scheme': ...",None,None,None,None,None,None,None,None,None,None,dedup_wf_002::00cece367574590aa05deb123340c6e6,"[oai:sedici.unlp.edu.ar:10915/94520, 50|od____...",None,None,None,"{'citationImpact': {'citationClass': 'C5', 'ci...",None,"[{'acronym': 'UNLP', 'id': 'openorgs____::40b9...",None,[{'key': 'opendoar____::dc6e224a8d74ce03bf3011...,"[{'accessRight': None, 'alternateIdentifiers':...",True,False,openorgs____::40b9f835648a3e0d057d6917dd7e54d5,2025-09-04
299,"[{'fullName': 'TomÃ¡s, Facundo', 'name': 'Facu...",gold,False,publication,"{'code': 'eng', 'label': 'English'}","[{'code': 'AR', 'label': 'Argentina', 'provena...","[{'provenance': None, 'subject': {'scheme': 'k...",Guerra Civil española y carteles de propaganda...,None,[El cartelismo durante la Guerra Civil español...,2006-01-01,Universidad Nacional de La Plata,

In [3]:
def _pick_load_dt(df: pd.DataFrame):
    if 'load_datetime' not in df.columns or df['load_datetime'].isna().all():
        return date.today()
    vals = df['load_datetime'].dropna()
    if vals.nunique() == 1:
        return vals.iloc[0]
    return pd.to_datetime(vals).max().date()


In [4]:

load_dt = _pick_load_dt(df)


## Paso 1: Convierto tipos y selecciono columnas con cardinalidad 1 con respecto a cada research product
+ info en https://graph.openaire.eu/docs/data-model/entities/research-product

In [5]:
def openaire_land_researchproduct(df: pd.DataFrame)-> pd.DataFrame:

    load_dt = _pick_load_dt(df)

    expected_columns = [
        'id',
        'openAccessColor',
        'publiclyFunded',
        'type',
        'language',
        'country',
        'mainTitle',
        'description',
        'publicationDate',
        'format',
        'bestAccessRight',
        'indicators',
        'isGreen',
        'isInDiamondJournal',
        'publisher',
        'source',
        'container',
        'contributor',
        'contactPerson',
        'coverage',
        'contactPerson',
        'embargoEndDate',
        'dateOfCollection',        
    ]

    # Agregar columnas faltantes con NaN
    for col in expected_columns:
        if col not in df.columns:
            df[col] = pd.NA

    df = df.convert_dtypes()

    df_researchproduct = df[expected_columns].copy()
    df_researchproduct.reset_index(drop=True, inplace=True)

    # language
    df_researchproduct['language'] = df_researchproduct['language'].apply(
        lambda x: x if isinstance(x, dict) else {}
    )
    df_researchproduct['language_code'] = df_researchproduct['language'].apply(lambda x: x['code'])
    df_researchproduct['language_label'] = df_researchproduct['language'].apply(lambda x: x['label'])

    
    ## bestAccessRight
    df_researchproduct['bestAccessRight_label'] = df['bestAccessRight'].apply(lambda x: x['label'] if x else None)
    df_researchproduct['bestAccessRight_scheme'] = df['bestAccessRight'].apply(lambda x: x['scheme'] if x else None)

    ## indicators
    df_indicators = pd.json_normalize(df['indicators']).reset_index(drop=True)
    
    indicators_expected_columns = [
        "citationImpact.citationClass",
        "citationImpact.citationCount",
        "citationImpact.impulse",
        "citationImpact.impulseClass",
        "citationImpact.influence",
        "citationImpact.influenceClass",
        "citationImpact.popularity",
        "citationImpact.popularityClass",
        "usageCounts.downloads",
        "usageCounts.views",
    ]

    # Agregar columnas para indicators y faltantes con NaN
    for col in indicators_expected_columns:
        if col not in df_indicators.columns:
            df_indicators[col] = pd.NA

    df_researchproduct = pd.concat([df_researchproduct.drop(columns=['indicators']).reset_index(drop=True), df_indicators], axis=1)

    # TODO country
    # TODO description
    # TODO format
    # TODO instance
    # TODO container
    # TODO contributor
    # TODO contactPerson
    # TODO coverage

    ## drop de columnas procesadas en otros df
    df_researchproduct.drop(columns=[
        'country', 'bestAccessRight', 
        'language', 'format',  
        'container', 'source', 'description',
        'contributor', 'contactPerson', 'coverage'
        ], inplace=True)

    df_researchproduct['load_datetime'] = load_dt

    return df_researchproduct


In [6]:
df_researchproduct = openaire_land_researchproduct(df)

In [7]:
df_researchproduct

,id,openAccessColor,publiclyFunded,type,mainTitle,publicationDate,isGreen,isInDiamondJournal,publisher,embargoEndDate,dateOfCollection,language_code,language_label,bestAccessRight_label,bestAccessRight_scheme,usageCounts,citationImpact.citationClass,citationImpact.citationCount,citationImpact.impulse,citationImpact.impulseClass,citationImpact.influence,citationImpact.influenceClass,citationImpact.popularity,citationImpact.popularityClass,usageCounts.downloads,usageCounts.views,load_datetime
0,4dc99724cf04::319dc88111c9b2d6021228590e79130a,gold,False,publication,Dicistrovirus from the pollinator community fo...,2022-08-31,False,False,"Universidad Nacional Mayor de San Marcos, Facu...",None,None,spa,Spanish; Castilian,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.635068e-09,C5,2.236927e-09,C5,NaN,NaN,2025-09-04
1,4dc99724cf04::95ea5df70a451a0487e051faa6c0a646,gold,False,publication,Variability in the growth rates of Saanen kids...,2023-12-18,False,False,"Universidad Nacional Mayor de San Marcos, Facu...",None,None,spa,Spanish; Castilian,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.635068e-09,C5,2.588463e-09,C5,NaN,NaN,2025-09-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,dedup_wf_002::00cece367574590aa05deb123340c6e6,<NA>,False,publication,Efecto de la concentración del silano MTMO sob...,2017-01-01,True,False,<NA>,None,None,esl/spa,Spanish,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.635068e-09,C5,1.121260e-09,C5,NaN,NaN,2025-09-04
299,dedup_wf_002::00cf1fe88397a2c3ce9d41fe616b29cf,gold,False,publication,Guerra Civil española y carteles de propaganda...,2006-01-01,True,False,Universidad Nacional de La Plata,None,None,eng,English,OPEN,http://vocabularies.coar-repositories.org/docu...,NaN,C5,0.0,0.0,C5,2.635068e-09,C5,3.675335e-10,C5,NaN,NaN,2025-09-04
